In [33]:
import requests
import pandas as pd

#설정상수
OC = 'topaz11100'
TYPE = 'JSON'

#본문조회상수
TEXT_URL = 'http://www.law.go.kr/DRF/lawService.do?target=prec'
TEXT_PARAM = {
    'OC': OC,
    'ID': '241881',
    'type': TYPE
}
TEXT_KEY = ['판시사항', '참조판례', '사건종류명', '판결요지', '참조조문', '선고일자', '법원명', '사건명', '판례내용', '사건번호', '사건종류코드', '판례정보일련번호', '선고', '판결유형', '법원종류코드']

#목록조회상수
LIST_URL = 'http://www.law.go.kr/DRF/lawSearch.do?target=prec'
LIST_PARAM = {
    'OC': OC,
    'type': TYPE,
    'display': 100
}
LIST_KEY = ['키워드', 'page', 'target', 'prec', 'totalCnt', 'section']
LIST_PREC_KEY = ['id', '사건번호', '데이터출처명', '사건종류코드', '사건종류명', '선고', '선고일자', '판례일련번호', '판결유형', '법원종류코드', '법원명', '판례상세링크', '사건명']

#요청, 응답을 딕셔너리로 반환함
def api_to_dict(url, param):
    response = requests.get(url, params=param)
    response.raise_for_status()
    dict_data = next(iter(response.json().values()))
    if type(dict_data) is not dict:
        raise TypeError(f"딕셔너리가 아님 어쩌면 판례없음")
    return dict_data

text_data, text_row = pd.DataFrame(columns=TEXT_KEY), []
list_data, list_row = pd.DataFrame(columns=LIST_PREC_KEY), []

def merge_text_row_data(text_data, text_row):
    return pd.concat([text_data, pd.DataFrame(text_row)], ignore_index=True)
def merge_list_row_data(list_data, list_row):
    return pd.concat([list_data, pd.DataFrame(list_row)], ignore_index=True)

In [34]:
#id를 넣은 파라미터 생성
def make_id_param(id):
    result = TEXT_PARAM.copy()
    result.update({'ID' : id})
    return result

#판례번호에 맞는 판례가 있으면 딕셔너리, 없으면 스트링
text_row.append(api_to_dict(TEXT_URL, make_id_param('179471')))
text_data = merge_text_row_data(text_data, text_row)

In [35]:
#검색설정
def make_query_param(query, curt=''):
    result = LIST_PARAM.copy()
    result.update({'query' : query})
    if curt:
        result.update({'curt' : curt})
    return result

list_row.append(api_to_dict(LIST_URL, make_query_param('이혼', '서울가정법원'))['prec'][1])
list_data = merge_list_row_data(list_data, list_row)

In [36]:
text_data.to_csv("본문테스트.csv", encoding='cp949')

In [37]:
list_data.to_csv("목록테스트.csv", encoding='cp949')